In [7]:
import cv2
import numpy as np
# taking image path or name from user
path = input('Enter Image path to recognize defects in: ')
# Read image
img = cv2.imread(path)
img = cv2.imread(path,cv2.IMREAD_GRAYSCALE)

_,binary = cv2.threshold(img, 128, 255, cv2.THRESH_BINARY)

# clean up
ke = np.ones((5,5), dtype=np.uint8)
dst = cv2.morphologyEx(binary,cv2.MORPH_CLOSE,ke)
dst = cv2.bitwise_not(dst)

# detecting contours
cont, hier = cv2.findContours(dst, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE) 
areas = []
for i in range (len(cont)):
    areas.append(cv2.contourArea(cont[i]))

# storing indices of coutours with spades 
spadeIndex = []
for i in range(len(areas)):
    if(areas[i]> 7000 and areas[i]<9000):
        spadeIndex.append(i)

# storing contour information on spade indices
spadeContours = []
for i in range(len(spadeIndex)):
    spadeContours.append(cont[spadeIndex[i]])
    
dist1 = []
dist2 = []
dist3 = []
for i in range(len(spadeContours)):
    dist1.append(cv2.matchShapes(spadeContours[28],spadeContours[i],cv2.CONTOURS_MATCH_I1,0))
    dist2.append(cv2.matchShapes(spadeContours[28],spadeContours[i],cv2.CONTOURS_MATCH_I2,0))
    dist3.append(cv2.matchShapes(spadeContours[28],spadeContours[i],cv2.CONTOURS_MATCH_I3,0))

correctSpades = []
incorrectSpades = []
for i in range(len(dist1)):
    if (dist2[i]<1):
        correctSpades.append(spadeContours[i])
    else:
        incorrectSpades.append(spadeContours[i])

img3 = img.copy();
img3 = cv2.cvtColor(img3, cv2.COLOR_GRAY2RGB)
for i in range(len(incorrectSpades)):
    img3 = cv2.drawContours(img3, incorrectSpades, -1, (0,0,255),-1)
    
# scaling the image
scale_percent = 40 # percent of original size
width = int(img3.shape[1] * scale_percent / 100)
height = int(img3.shape[0] * scale_percent / 100)
dim = (width, height)
  
# resize image
img3 = cv2.resize(img3, dim, interpolation = cv2.INTER_AREA)
cv2.imwrite("spade-terminal-output.png", img3)
cv2.imshow("Defects", img3)
cv2.waitKey()
cv2.destroyAllWindows()

Enter Image path to recognize defects in: spade-terminal.png
